In [4]:
search_for = 124
start_from = 0
threads = 10
thread = 6
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0024390220642089844
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 75
all cases: 621252555
type: [1, 1, 1, 1, 124] 124
cases of this type: 236421376
100000 29.01334583053712
200000 50.82087096487994
300000 47.40021074892905
400000 46.7863187204416
500000 49.3691857756914
600000 47.79480523949751
700000 46.53740337759655
800000 46.58071709593621
900000 46.7022599452144
1000000 16.14914296591648
1100000 47.10528838272621
1200000 46.03100143986465
1300000 45.101183470632506
1400000 46.10744449592222
1500000 48.26328372635236
1600000 46.02347401541224
1700000 45.99926037890955
1800000 46.318236675631844
1900000 40.35981627168606
2000000 22.502817762880504
2100000 43.98860400220377
2200000 43.30048758437909
2300000 41.803396756758815
2400000 44.008548006157724
2500000 43.677582234726444
2600000 43.47327137510994
2700000 42.134181984778465
2800000 42.316148765669226
2900000 18.14272336530371
3000000 41.1851183535

29700000 41.706366760290216
29800000 40.72880900443834
29900000 40.977411685089926
30000000 43.66223514472715
30100000 43.762697897687474
30200000 44.32303716455686
30300000 27.473246336637825
30400000 38.65394183316546
30500000 45.41699228570441
30600000 45.97331344550172
30700000 46.580243407214454
30800000 20.397781256291772
30900000 46.46419411559979
31000000 47.75738535967648
31100000 45.55653177978007
31200000 45.82840695794112
31300000 42.771251029496305
31400000 42.53559002443727
31500000 17.252503321630588
31600000 41.66382935229959
31700000 41.663270685501864
31800000 44.029443017745756
31900000 44.938801392907735
32000000 46.90960256416422
32100000 47.16987623181626
32200000 24.998767330864162
32300000 43.17180706469691
32400000 47.164054762240404
32500000 45.23053107725313
32600000 44.11211997985121
32700000 26.652178996714333
32800000 38.439173592568686
32900000 46.587575105530156
33000000 42.15797308219438
33100000 41.67962554354547
33200000 40.078390819577024
33300000 38

59800000 36.51209959673378
59900000 36.73829267954017
60000000 37.0963614678027
60100000 14.70200542622725
60200000 37.626769559447425
60300000 36.03664050853489
60400000 37.354933498858514
60500000 36.82754457015393
60600000 19.769345293676853
60700000 38.73431619750479
60800000 40.13755535896095
60900000 39.73832937650785
61000000 42.94932186268822
61100000 42.469911423534725
61200000 41.69343438269891
61300000 41.08293558912363
61400000 41.26514371428222
61500000 32.33978212337673
61600000 24.58216540758581
61700000 37.03753276149043
61800000 39.63740847563374
61900000 36.82909614204564
62000000 17.446161318160936
62100000 34.73819569484971
62200000 38.6417387806493
62300000 36.06482901444295
62400000 36.366684609877986
62500000 21.088761699117633
62600000 39.63671852664169
62700000 39.945472211113604
62800000 41.508382281170746
62900000 41.6407020899163
63000000 41.12106334530146
63100000 40.64345282553801
63200000 40.396080295076416
63300000 37.993841268484545
63400000 38.08501241

89900000 35.68266283817676
90000000 35.077282068726674
90100000 34.692348142374044
90200000 35.08764237380271
90300000 34.07301015999364
90400000 16.603723699347327
90500000 31.930399794967126
90600000 15.692896006553383
90700000 31.580174419753718
90800000 26.449089809264024
90900000 25.657023941170923
91000000 34.68941173721482
91100000 34.20021246614705
91200000 34.65158071664946
91300000 34.62788878883847
91400000 36.4849997100665
91500000 37.57431241855144
91600000 39.41944890772356
91700000 39.61085197327191
91800000 37.227585102355185
91900000 34.79368481545673
92000000 35.8117543095276
92100000 35.66334226683504
92200000 34.966666545061095
92300000 15.803343336081996
92400000 35.65860082586202
92500000 16.54228973851869
92600000 31.06182101807253
92700000 23.532215114790876
92800000 28.524491126542202
92900000 34.48216696877242
93000000 35.29581794939635
93100000 35.23490479051772
93200000 34.78619703647405
93300000 37.703283724680006
93400000 39.372637883928284
93500000 37.950

119200000 3.691266513325963
119300000 3.760319979006194
119400000 3.7763319476245942
119500000 3.7673630791206727
119600000 3.7498907017371588
119700000 3.709693934458633
119800000 3.7337756965693267
119900000 3.7122505074164835
120000000 3.700549524032275
120100000 3.691381993602027
120200000 3.6991075138055747
120300000 3.7116300041105155
120400000 3.664325490811189
120500000 3.6954504297235444
120600000 3.6945692125327487
120700000 3.7529773387399343
120800000 3.9668377618086867
120900000 3.8321714687306883
121000000 3.8334109586538836
121100000 3.8401207040254572
121200000 3.8079826728524564
121300000 3.7714700406638415
121400000 3.806623173266941
121500000 3.823680881509572
121600000 3.7893211144426266
121700000 3.7910231846467357
121800000 3.758477215966523
121900000 3.823122229213622
122000000 3.815971256965068
122100000 3.8250177449900504
122200000 3.871262775930057
122300000 3.879254042536113
122400000 3.8389988250139853
122500000 3.8412717516386707
122600000 3.806498607432952

...

493200000 3.7741001751475034
493300000 3.5387399471425156
493400000 1.4247359893546239
493500000 3.3395705455783404
493600000 2.9580838332126342
493700000 3.5244348066530957
493800000 3.369790250296355
493900000 3.601256382302619
494000000 3.632755659950574
494100000 4.031044264729282
494200000 3.7523687490427826
494300000 2.7725488972377015
494400000 2.435249792880545
494500000 3.1733189459573325
494600000 2.3694517208939128
494700000 3.353904691794336
494800000 3.594195061076681
494900000 4.008988103223155
495000000 3.88516944715334
495100000 3.7782418920257723
495200000 3.677521523303075
495300000 1.2585664100314378
495400000 3.2613649653262957
495500000 2.2106751872890325
495600000 3.9212357743099333
495700000 3.472332871780065
495800000 3.488733450497571
495900000 3.411729545963913
496000000 3.4003837041898137
496100000 3.7630619292389254
496200000 2.669667192723741
496300000 2.667163113514036
496400000 2.5221526950273385
496500000 3.056866373995811
496600000 3.282033438543648
496

521200000 2.9187789836751326
521300000 2.8906513368918234
521400000 2.9646786427125336
521500000 2.9843540424214403
521600000 3.0697964618482025
521700000 2.999365678550631
521800000 3.085327450035632
521900000 3.0803404699200163
522000000 3.2436251170826056
522100000 3.105171359629303
522200000 3.18170628478134
522300000 3.134875512530446
522400000 3.220455469843967
522500000 3.174195275427537
522600000 3.142146162962824
522700000 3.2076109964784845
522800000 3.2454633537103
522900000 3.134378835355908
523000000 3.20816395658652
523100000 3.0749933785532084
523200000 3.243926943924437
523300000 3.136703665605727
523400000 3.1706426753295958
523500000 2.8225333544181885
523600000 2.736964944561458
523700000 2.804272131181641
523800000 3.010069901971012
523900000 3.00417497922193
524000000 3.042644841118985
524100000 2.8107642406499584
524200000 3.0213834535661803
524300000 2.9847765130185686
524400000 3.6760943372381325
524500000 3.417464456996146
524600000 3.6463892094699974
524700000

548000000 1.7461383368661305
548100000 2.7489708442992664
548200000 3.0791895338217152
548300000 0.9949173081348374
548400000 0.5367507280483842
548500000 0.533191023132109
548600000 0.5327396662968132
548700000 0.5916205982204974
548800000 0.5588287624787821
548900000 0.5603000324880315
549000000 0.5496067445933819
549100000 0.5820344896903965
549200000 0.5534997776959174
549300000 0.5508539453793964
549400000 0.548858504072451
549500000 0.5507742325380974
549600000 0.5513554992034734
549700000 0.5505636949736542
549800000 0.5479704366707439
549900000 0.546805649051696
550000000 0.5466848470282852
550100000 0.5465548819175462
550200000 0.5417228651217321
550300000 0.538628700734208
550400000 0.5386819061793917
550500000 0.5411863744197786
550600000 0.5552595223714113
550700000 3.4411500829198856
550800000 3.232099259762476
550900000 1.3806331198973456
551000000 3.2490961023131475
551100000 3.3781343878792827
551200000 2.798029894705319
551300000 2.335879681881067
551400000 1.505693881

576300000 1.3029854270100296
576400000 1.2803479140527547
576500000 1.0653901314368877
576600000 0.7316180794570744
576700000 1.139065220355789
576800000 1.319064371381816
576900000 1.0496291656650008
577000000 1.2199419465808736
577100000 0.5988578511565493
577200000 1.3396694592700202
577300000 1.306147988175644
577400000 0.9616712775241204
577500000 1.1589177705280782
577600000 0.6736750589688851
577700000 1.3255065163953899
577800000 1.1272869487943749
577900000 1.0286783739868535
578000000 0.9302431671067345
578100000 0.845326471892188
578200000 1.2865095880742536
578300000 0.9356044520603518
578400000 1.2109125634245572
578500000 0.7347328650256925
578600000 1.0660573014043537
578700000 1.267168804861347
best so far: 0
type: [1, 2, 1, 31, 2] 124
cases of this type: 953312
578800000 0.4113019122985568
578900000 0.27327495925133094
579000000 0.2732861404437224
579100000 0.2740623937804732
579200000 0.27387899699190255
579300000 0.27145328886681797
579400000 0.26980748322029574
5795

599700000 0.1661252628603975
599800000 0.412319410822524
599900000 0.7660879863415095
600000000 0.882658874333471
600100000 0.8741424110615718
600200000 0.5825598060717351
600300000 0.8409643648950259
600400000 0.5083251845434805
600500000 0.7966107620072398
600600000 0.7488275774619183
600700000 0.6324636373066372
600800000 0.8341783164842527
600900000 0.5369199153782725
601000000 0.7909632755968836
601100000 0.6025057094444036
601200000 0.7059804773514469
601300000 0.7040135481603973
601400000 0.5762695883769228
601500000 0.7874858066893021
601600000 0.5443046511510512
601700000 0.7458733966722157
601800000 0.6224225361947715
601900000 0.6180194687374234
602000000 0.7194270313087072
602100000 0.5615825102397501
602200000 0.8028146640017761
602300000 0.5753458984036545
602400000 0.6871967542083363
602500000 0.6942647270132866
602600000 0.5602052967402769
602700000 0.7302461846851109
602800000 0.595433466322932
602900000 0.6933609154093398
603000000 0.5568414953835508
603100000 0.66102